# **I. Install transformers Datasets to get wikiSQL Dataset**




In [4]:
!pip install -q -U datasets > /dev/null

In [1]:
# install transfoermers and sentencepiece (unsupervised text tokenizer and detokenizer)
!pip install transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 103.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 122.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.8 MB/s eta 0:00:00


In [2]:
!pip install deep-translator

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 KB 12.7 MB/s eta 0:00:00
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [5]:
from transformers import AutoModelWithLMHead, AutoTokenizer
from deep_translator import GoogleTranslator
from datasets import load_dataset
import random, warnings
import regex as re
warnings.filterwarnings("ignore")

# **II. Load T5-base model fine-tuned on WikiSQL**

In [6]:
tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-wikiSQL")
model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-wikiSQL")

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


Downloading:   0%|          | 0.00/1.19G [00:00<?, ?B/s]

In [7]:
def get_sql(query):
  ################################################
  # this function take an english query sentence #
  # and return an SQL Query                      #
  ################################################
  input_text = "translate English to SQL: %s </s>" % query
  features = tokenizer([input_text], return_tensors='pt', truncation=True)
  output = model.generate(input_ids=features['input_ids'], 
              attention_mask=features['attention_mask'])
  
  return tokenizer.decode(output[0])

# **III. Make predictions on WikiSQL Validation Set**

In [ ]:
valid_dataset = load_dataset('wikisql', split='validation')

Generating test split:   0%|          | 0/15878 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8421 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/56355 [00:00<?, ? examples/s]

Dataset wikisql downloaded and prepared to /root/.cache/huggingface/datasets/wikisql/default/0.1.0/7037bfe6a42b1ca2b6ac3ccacba5253b1825d31379e9cc626fc79a620977252d. Subsequent calls will reuse this data.


In [ ]:
len(valid_dataset)

8421

In [ ]:
# Get SQl keywords in validation dataset
sql_set = set()
for s in valid_dataset:
  s_temp = s['sql']['human_readable']
  l_sql = re.findall(r'\b[A-Z]+(?:\s+[A-Z]+)*\b', s_temp)
  print(l_sql)
  sql_set.update(set(l_sql))

Streaming output truncated to the last 5000 lines.
['SELECT COUNT', 'FROM', 'WHERE', 'AND']
['SELECT AVG', 'FROM', 'WHERE']
['SELECT COUNT', 'FROM', 'WHERE']
['SELECT', 'FROM', 'WHERE']
['SELECT', 'FROM', 'WHERE']
['SELECT', 'FROM', 'WHERE', 'AND']
['SELECT', 'FROM', 'WHERE', 'AND']
['SELECT MIN', 'FROM', 'WHERE']
['SELECT MIN', 'FROM', 'WHERE']
['SELECT MAX', 'FROM', 'WHERE']
['SELECT MIN', 'FROM', 'WHERE']
['SELECT MAX', 'FROM', 'WHERE', 'AND', 'AND']
['SELECT MIN', 'FROM', 'WHERE']
['SELECT MIN', 'FROM', 'WHERE', 'AND']
['SELECT SUM', 'FROM', 'WHERE', 'AND', 'AND']
['SELECT', 'FROM', 'WHERE', 'AND']
['SELECT MIN', 'FROM', 'WHERE']
['SELECT', 'FROM', 'WHERE']
['SELECT', 'FROM', 'WHERE', 'AND', 'AND']
['SELECT', 'FROM', 'WHERE', 'AND']
['SELECT', 'FROM', 'WHERE']
['SELECT', 'FROM', 'WHERE']
['SELECT AVG', 'FROM', 'WHERE', 'AND', 'AND', 'AND']
['SELECT AVG', 'FROM', 'WHERE', 'AND']
['SELECT MAX', 'FROM', 'WHERE', 'AND']
['SELECT AVG', 'FROM', 'WHERE', 'AND', 'AND']
['SELECT COUNT', 'FR

In [ ]:
sql_set

{'A',
 'A FROM',
 'ABS',
 'AG',
 'AM',
 'AND',
 'AND A',
 'AND B',
 'AND BR',
 'AND CONCACAF',
 'AND D',
 'AND E',
 'AND ERP W',
 'AND F',
 'AND FA',
 'AND FPU',
 'AND GDP',
 'AND GP',
 'AND H',
 'AND HDTV',
 'AND IATA',
 'AND ICAO',
 'AND ICFC',
 'AND IHSAA',
 'AND LD',
 'AND LNER',
 'AND MLS',
 'AND N',
 'AND NFL',
 'AND NGC',
 'AND NHL',
 'AND PSD',
 'AND QUART',
 'AND TD',
 'AND U',
 'AND US',
 'AND VIN',
 'AVUS',
 'AZ',
 'B',
 'B FROM',
 'BARB',
 'BBC',
 'BC',
 'BCP',
 'BS',
 'BSI',
 'BW',
 'C',
 'C FROM',
 'CBN TV',
 'CBS AND',
 'CC',
 'CH',
 'CHAN',
 'CITV',
 'CMA',
 'COM',
 'COM FROM',
 'COMMAND',
 'CONCACAF',
 'CSL FROM',
 'CT',
 'D',
 'D FROM',
 'DFL',
 'DR',
 'DVR',
 'DWCI',
 'DWEC',
 'E',
 'EFL',
 'ERP',
 'ESP',
 'ET',
 'EU',
 'EXE',
 'F',
 'FC',
 'FIBA',
 'FK',
 'FL',
 'FL AND',
 'FROM',
 'G',
 'G FROM',
 'GAU',
 'GB',
 'GDP',
 'GNIS',
 'GP',
 'GP FROM',
 'GP H',
 'GPI',
 'GPU',
 'GRC',
 'GRT FROM',
 'GS',
 'GT',
 'GW',
 'H',
 'HD',
 'HDI',
 'HK',
 'HK FROM',
 'HP',
 'HPI'

In [ ]:
# Sample Validation Dataset
valid_dataset[0]

{'phase': 1,
 'question': 'What position does the player who played for butler cc (ks) play?',
 'table': {'header': ['Player',
   'No.',
   'Nationality',
   'Position',
   'Years in Toronto',
   'School/Club Team'],
  'page_title': 'Toronto Raptors all-time roster',
  'page_id': '',
  'types': ['text', 'text', 'text', 'text', 'text', 'text'],
  'id': '1-10015132-11',
  'section_title': 'L',
  'caption': 'L',
  'rows': [['Antonio Lang',
    '21',
    'United States',
    'Guard-Forward',
    '1999-2000',
    'Duke'],
   ['Voshon Lenard', '2', 'United States', 'Guard', '2002-03', 'Minnesota'],
   ['Martin Lewis',
    '32, 44',
    'United States',
    'Guard-Forward',
    '1996-97',
    'Butler CC (KS)'],
   ['Brad Lohaus', '33', 'United States', 'Forward-Center', '1996', 'Iowa'],
   ['Art Long',
    '42',
    'United States',
    'Forward-Center',
    '2002-03',
    'Cincinnati'],
   ['John Long', '25', 'United States', 'Guard', '1996-97', 'Detroit'],
   ['Kyle Lowry', '3', 'United Sta

In [ ]:
for idx in random.sample(range(len(valid_dataset)), 250):
    print(f"Text: {valid_dataset[idx]['question']}")
    print(f"Pred SQL: {get_sql(valid_dataset[idx]['question'])}")
    print(f"True SQL: {valid_dataset[idx]['sql']['human_readable']}\n")

Text: Which district has John Culpepper (f) as the vacator?
Pred SQL: <pad> SELECT District FROM table WHERE Vacator = John Culpepper (
True SQL: SELECT District FROM table WHERE Vacator = John Culpepper (F)

Text: what's the won with try bonus being 8
Pred SQL: <pad> SELECT Won FROM table WHERE Try bonus = 8</s>
True SQL: SELECT Won FROM table WHERE Try bonus = 8

Text: What is the area when the Iga name is Ahoada East?
Pred SQL: <pad> SELECT Area (km2) FROM table WHERE Iga name = Aho
True SQL: SELECT MAX Area (km 2 ) FROM table WHERE LGA Name = Ahoada East

Text: Which bike did Jiri Drazdak ride when he had a grid number larger than 14 and less than 22 laps?
Pred SQL: <pad> SELECT Bike FROM table WHERE Grid > 14 AND Laps <unk>
True SQL: SELECT Bike FROM table WHERE Grid > 14 AND Laps < 22 AND Rider = jiri drazdak

Text: What is the word count that is named omega dot?
Pred SQL: <pad> SELECT Word count FROM table WHERE Name = omega dot</s>
True SQL: SELECT Word # FROM table WHERE Name 

# **Make predictions on FR Query translated to EN**

In [35]:
def sql_en2fr(s):
  ###############################################
  # This function take an english SQL Query and #
  # return its corespond in French              #
  ###############################################

  # Remove '<pad>' and '</s>' (added by the model)
  en_sql = re.sub("<pad>", "", s)
  en_sql = re.sub("</s>", "", en_sql)

  # Find all english SQL Query values
  fr_sql = ''
  select_count_from = re.findall('SELECT COUNT' + '(.*?)' + 'FROM', en_sql)
  select_min_from = re.findall('SELECT MIN' + '(.*?)' + 'FROM', en_sql)
  select_max_from = re.findall('SELECT MAX' + '(.*?)' + 'FROM', en_sql)
  select_avg_from = re.findall('SELECT AVG' + '(.*?)' + 'FROM', en_sql)
  select_from = re.findall('SELECT' + '(.*?)' + 'FROM', en_sql)
  where_operator  = re.findall('WHERE' + '(.*?)' + '[<-=->]', en_sql)

  and_operator = re.findall('AND' + '(.*?)' + '[<-=->]', en_sql)
  operators = re.findall('[<-=->]', en_sql)

  operator_end = ''
  if len(operators) > 0:
    operator_end = en_sql[en_sql.rfind(operators[-1])+2:]

  # Translate them to french language
  # Filling the SQL structure
  if select_count_from :
    fr_sql += 'SELECT COUNT(' + GoogleTranslator(source='auto', target='fr').translate(select_count_from[0]) + ') FROM table '
  elif  select_min_from:
    fr_sql += 'SELECT MIN(' + GoogleTranslator(source='auto', target='fr').translate(select_min_from[0]) + ') FROM table '
  elif  select_max_from:
    fr_sql += 'SELECT MAX(' + GoogleTranslator(source='auto', target='fr').translate(select_max_from[0]) + ') FROM table '
  elif  select_avg_from:
    fr_sql += 'SELECT AVG(' + GoogleTranslator(source='auto', target='fr').translate(select_avg_from[0]) + ') FROM table '
  elif  select_from:
    fr_sql += 'SELECT ' + GoogleTranslator(source='auto', target='fr').translate(select_from[0]) + ' FROM table '
  
  if where_operator:
    fr_sql += 'WHERE ' + GoogleTranslator(source='auto', target='fr').translate(where_operator[0]) + ' ' + operators[0] + ' '
  else:
    fr_sql = fr_sql.strip()
  i = 1
  if and_operator:
    for tok in and_operator:
      fr_sql += 'AND ' + GoogleTranslator(source='auto', target='fr').translate(tok) + ' ' + operators[i] + ' '
      i += 1

  if  where_operator or and_operator:
    fr_sql +=  operator_end 

  fr_sql += ';'  

  return fr_sql

In [36]:
# Tesing with this french sentences
to_translate = ["Combien de personnes nommés Hassan",
                "Sélectionner les magasins ayant un chiffre d'affaire supérieur à 100000",
                "Donner moi le max et la moyenne des salaires des employés",
                "Afficher le titre et l'auteur de l'ouvrage numero 4",
                "Quel'est le max des salaire des employés",
                "Quel'est la note moyenne des etudiants pour module NLP"
                ]

for i in range(0, len(to_translate)):
  translated = GoogleTranslator(source='auto', target='en').translate(to_translate[i])
  print("French Query : ", to_translate[i])
  print("Translated to english Query : ",  translated)
  print("SQL Query : ", sql_en2fr(get_sql(translated)))
  print("########################################################################################")
  print()

French Query :  Combien de personnes nommés Hassan
Translated to english Query :  How many people named Hassan
SQL Query :  SELECT COUNT(Nom) FROM table WHERE Nom = Hassan;
########################################################################################

French Query :  Sélectionner les magasins ayant un chiffre d'affaire supérieur à 100000
Translated to english Query :  Select stores with a turnover greater than 100000
SQL Query :  SELECT Boutique FROM table WHERE Chiffre d'affaires > 100000;
########################################################################################

French Query :  Donner moi le max et la moyenne des salaires des employés
Translated to english Query :  Give me the max and average salaries of employees
SQL Query :  SELECT MAX(Salaire moyen) FROM table;
########################################################################################

French Query :  Afficher le titre et l'auteur de l'ouvrage numero 4
Translated to english Query :  Display 